#### Notebook to feature engineer fraction of time formaldehyde was above 75th percentile

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2006','2006_multi_level_26_variables.nc')
ml_26_variables = xr.open_dataset(PATH_1) ## multilevel 26 variables
ml_26_variables = ml_26_variables.squeeze(dim="model_level") ## remove the model_level dimension
ml_26_variables=ml_26_variables.drop_vars('model_level') ## the dropped dimension becomes a column, so remove that
ml_26_variables['longitude'] = ml_26_variables['longitude']-360 ## necessary because of longitude data in multi-level case
formal=ml_26_variables['hcho'] ## get only formaldehyde variable
formal

<xarray.DataArray 'hcho' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2006-01-01 ... 2006-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/31)
    GRIB_paramId:                             210124
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           hcho
    GRIB_units:                               kg kg**-1
    long_name:                                Formaldehyde
    units:                                    kg kg**-1
    standard_name:                            mass_fraction_of_formaldehyde_i...

In [3]:
formal.to_dataframe()

hcho
valid_time          latitude longitude                              
2006-01-01 00:00:00 49.25    -124.849999999999994  0.000000000339665
                             -124.099999999999994  0.000000000618417
                             -123.349999999999994  0.000000000981543
                             -122.599999999999994  0.000000000818474
                             -121.849999999999994  0.000000000405187
...                                                              ...
2006-12-31 21:00:00 24.50    -70.100000000000023   0.000000000391608
                             -69.350000000000023   0.000000000392353
                             -68.600000000000023   0.000000000372524
                             -67.850000000000023   0.000000000342805
                             -67.100000000000023   0.000000000330946

[7743840 rows x 1 columns]

In [4]:
PATH_2=os.path.join('..','CAMS_79_variables_2006.pkl')
get_data = pd.read_pickle(PATH_2) ## load the data with all variables
get_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '..\\CAMS_79_variables_2006.pkl'

In [ ]:
percentile_75 = get_data['hcho'].quantile(0.75) ## calculate 75th percentile
print(percentile_75)
threshold=percentile_75

In [ ]:
## use county shapefile from 2008 as the shapefile before 2008 could not be found
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2008_county_shapefile','tl_2008_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

In [ ]:
geometry_column=county_gdf['geometry']
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

In [ ]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time above percentile
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= formal.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['hcho'].values[i] > threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'formaldehyde above percentile':[frac_time]})


    return pm_above_df

In [ ]:
len(county_gdf)

In [ ]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

In [ ]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

In [ ]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop([ 'STATEFP','COUNTYFP','COUNTYNS','CNTYIDFP','NAMELSAD','LSAD','CLASSFP','MTFCC',
                                 'CSAFP','CBSAFP','METDIVFP','FUNCSTAT'], axis=1)
county_gdf

In [ ]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

In [ ]:
county_df=merged_df.dropna()
county_df.head()

In [ ]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

In [ ]:
county_var.to_pickle('formaldehyde_above_percentile.pkl')